In [1]:
# Get path to convergence

def get_X_prime(X, X_unique, D_unique, sigma):
    di = get_di(sigma, X_unique, X, D_unique)
    wi = get_wi(sigma, X_unique, X, D_unique)
    if len(get_Si_prime(X[2],di,X[4])) == 1:
        Si_prime = get_Si_prime(X[2],di,X[4])[0]
    else:
        prob = get_prob(get_Si_prime(X[2],di,X[4])[0], (get_Sj_prime(X[2], di, X[4],get_Si_prime(X[2],di,X[4])[0]) ) )
        obs_prob = np.random.binomial(1, prob, 1)[0]
        if obs_prob == 1:
            Si_prime = get_Si_prime(X[2],di,X[4])[0]
        else:
            Si_prime = get_Si_prime(X[2],di,X[4])[1]
            
            
    Sj_prime = get_Sj_prime(X[2],di,X[4],Si_prime)
    Li_prime = get_Li_prime(X[1],di,X[4],Si_prime)           
    Wi_prime = get_Wi_prime(player, wi,Li_prime , Si_prime, Sj_prime)
    dj_prime = X_unique[np.where(np.all(X_unique[:,:4] == [Wi_prime,Li_prime,Si_prime,Sj_prime] ,axis=1))[0][0]][4] # bring the dj from the X_unique matrix
    return np.array([Wi_prime,Li_prime, Si_prime, Sj_prime, dj_prime])

def get_X_path(time,X_unique, D_unique, sigma):
    path = np.zeros((time,len(X_unique[0])),dtype = np.int8 )
    if (get_X_prime(X_unique[0],X_unique, D_unique, sigma)[0:4] == [b,0,0,0]).all(): # if the equilibrium is reducible, starts from the rich side
        path[0] = [W_bar,0,0,0,0]
    else:
        path[0] = get_X_prime(X_unique[0],X_unique, D_unique, sigma)
    for t in range(1,time):
        path[t] = get_X_prime(path[t-1],X_unique, D_unique,sigma)
    return path 

def get_D_path(path,sigma,X_unique, D_unique):
    sigma_D = np.zeros((len(path),2),dtype = np.int8 )
    row = 0
    for i in path:
        sigma_D[row] = [get_di(results.sigma, X_unique, i[:4], D_unique),get_wi(sigma, X_unique, i, D_unique)]
        row += 1
    return sigma_D
        
def get_acc_path(path,sigma,X_unique, D_unique):
    acc = np.zeros(len(path),dtype = np.int8 )
    acc[0] = get_di(results.sigma, X_unique, path[0][:4], D_unique)
    row = 1
    for i in path[1:]:
        acc[row] = get_di(results.sigma, X_unique, i[:4], D_unique) + acc[row - 1]
        row += 1
    return acc
    
    
def get_paths(matrix, results, obs):
    path_X = get_X_path(obs,matrix.X_unique, matrix.D_unique,results.sigma)
    path_D = get_D_path(path_X,results.sigma, matrix.X_unique ,matrix.D_unique)
    acc = get_acc_path(path_X,results.sigma, matrix.X_unique ,matrix.D_unique)
    return path_X, path_D ,acc

In [ ]:
#Get the vector for plotting

def get_plot_vec(W_bar , X_unique , D_unique, sigma):
    vec = np.zeros([W_bar,9],dtype = np.half )
    vec[0] = [0,0,0,0,0,0,0,0,0] #initial vector
    for i in range(1, W_bar):
        vec[i,0] = i
        if [vec[i-1,0], 1, 1, 1, 1 ] in X_unique.tolist(): # Decison is given by a fixed liability level and a player who always receive a bribe
            vec[i,1] = get_di(sigma, X_unique, [vec[i-1,0],1, 1, 1, 1 ] ,D_unique)
        else:
            vec[i,1] = vec[i-1, 1]
        if vec[i-1,1] == 1: # L state evolves given the di last period
            if  vec[i-1,2]*delta + 1 < L_bar: # it goes up to L_bar
                vec[i,2] = vec[i-1,2]*(1-delta) + 1
            else:
                vec[i,2] = L_bar
        else:
            vec[i,2] = vec[i-1,2]*(1-delta)
        vec[i,3] = vec[i-1, 2]*alpha
        vec[i,4] = vec[i-1, 2]*alpha*beta
        vec[i,5] = vec[i-1, 2]*alpha*(1-beta)
        vec[i,6] = 0
        if [vec[i-1,0], 1, 1, 1, 1 ] in X_unique.tolist():
            vec[i,7] =  vec[i,0] - (get_wi(sigma, X_unique,  [vec[i-1,0],1, 1, 1] , D_unique))
        else:
            vec[i,7] = vec[i-1,7]
        if vec[i,1] == 1:
            vec[i,8] = vec[i-1,8] + vec[i,1]
        else:
            vec[i,8] = vec[i-1,8]
    return vec

# Create some mock data
def plot_individual_graph(vec):

    fig, ax = plt.subplots(1,2,figsize = (13,3))

    ax[1].grid(True)
    ax[1].set_xlabel('Wealth')
    ax[1].set_ylabel('Frequency', color='tab:red')
    ax[1].plot(vec[:,0], vec[:,3], label = 'Detections', color='tab:orange')
    ax[1].plot(vec[:,0], vec[:,4], label = 'Convictions' ,color='tomato')
    ax[1].plot(vec[:,0], vec[:,5], label = 'Acquitances' , color='tab:red')
    ax[1].tick_params(axis='y', labelcolor='tab:red')
    ax[1].set_ylim(0, 0.5)
    ax[1].set_xlim(3, W_bar)


    ax[0].grid(True)
    ax[0].set_ylabel('Quantity', color= 'black')  # we already handled the x-label with ax1
    ax[0].plot(vec[:,0], vec[:,7], label = 'Consume', color='grey')
    ax[0].plot(vec[:,0], vec[:,8], label = 'Cummulated Corruption', color='black')
    ax[0].plot(vec[:,0], vec[:,2], label = 'Liability for Corruption', color='black')
    ax[0].tick_params(axis='y', labelcolor='black')
    ax[0].set_ylim(0, W_bar )
    ax[0].set_xlim(4, W_bar)


    ax[0].legend(loc = 'upper left')
    ax[1].legend(loc = 'upper left')
    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    plt.savefig('./Plottings/Simple_{}_W_{}a_{}b_{}R_{}P_{}.pdf'.format(player,W_bar,alpha,beta,R,P))
    
    return plt.show()

        


In [ ]:
def get_heatmap_vec_pre(sigma, X_unique, D_unique):
    vec = np.zeros((L_bar+1,W_bar+1),dtype = np.int8 )
    for i in range(W_bar+1):
        for j in range(L_bar):
            if ([i,j,1,1] in X_unique[:,:4].tolist()):
                X = [i,j,1,1]
                vec[j,i] = get_di(sigma, X_unique, X, D_unique)
            elif [i,j,0,0] in X_unique[:,:4].tolist():
                X = [i,j,0,0]
                vec[j,i] = get_di(sigma, X_unique, X, D_unique)
            else:
                vec[j,i] = 10
    return vec

def get_heatmap_vec_post(sigma, X_unique, D_unique):
    vec = np.zeros((L_bar+1,W_bar+1),dtype = np.int8 )
    for i in range(W_bar+1):
        for j in range(L_bar+1):
            if ([i,j,2,2] in X_unique[:,:4].tolist()):
                X = [i,j,2,2]
                vec[j,i] = get_di(sigma, X_unique, X, D_unique)
            else:
                vec[j,i] = 10
    return vec
        

In [ ]:
from matplotlib.patches import Patch
from matplotlib.colors import ListedColormap 
from matplotlib.colors import BoundaryNorm 

def plot_heatmap(vec1,vec2):
    fig, ax = plt.subplots(2,1,figsize = (10,6))
    
    # Let's also design our color mapping: 1s should be plotted in blue, 2s in red, etc...
    col_dict_1={0:"tab:green",
                1:"salmon",
                2:"palegreen",
               10:"lightgrey",}
    col_dict_2={0:"salmon",
                2:"palegreen",
               10:"lightgrey"}

    # We create a colormar from our list of colors
    cm1 = ListedColormap([col_dict_1[x] for x in col_dict_1.keys()])
    cm2 = ListedColormap([col_dict_2[x] for x in col_dict_2.keys()])

    # prepare normalizer
    ## Prepare bins for the normalizer
    norm_bins_1 = np.sort([*col_dict_1.keys()]) + 0.5
    norm_bins_1 = np.insert(norm_bins_1, 0, np.min(norm_bins_1) - 1.0)

    
    ## Make normalizer and formatter
    norm_1 = BoundaryNorm(norm_bins_1, 4, clip=True)
    
    
    # prepare normalizer
    ## Prepare bins for the normalizer
    norm_bins_2 = np.sort([*col_dict_2.keys()]) + 0.5
    norm_bins_2 = np.insert(norm_bins_2, 0, np.min(norm_bins_2) - 1.0)

    
    ## Make normalizer and formatter
    norm_2 = BoundaryNorm(norm_bins_2, 3, clip=True)

    ax[0].pcolor(vec1, edgecolors='k', linewidths=.2, cmap = cm1, norm = norm_1)
    ax[0].set_title('Before Detection')
    ax[0].set_xlabel('Wealth')
    ax[0].set_ylabel('Liability')
    
    legend_elements_1 = [Patch(facecolor='lightgrey', edgecolor='k',
                         label='Non Reachable'),
                Patch(facecolor='salmon', edgecolor='k',
                         label='Corruption'),
                Patch(facecolor='tab:green', edgecolor='k',
                         label='No Corruption'),
                        Patch(facecolor= 'palegreen', edgecolor='k',
                         label='Reporting')]
    
    ax[0].legend(handles=legend_elements_1, bbox_to_anchor=(1.05, 1), loc='upper left')
    
    
    
    
    ax[1].pcolor(vec2, edgecolors='k', linewidths=.2, cmap = cm2,norm = norm_2)
    ax[1].set_xlabel('Wealth')
    ax[1].set_ylabel('Liability')
    ax[1].set_title('After Detection')
    
    legend_elements = [Patch(facecolor='lightgrey', edgecolor='k',
                         label='Non Reachable'),
                Patch(facecolor='salmon', edgecolor='k',
                         label='Not Admit Guilty'),
                Patch(facecolor='palegreen', edgecolor='k',
                         label='Admit Guilty')]
    
    ax[1].legend(handles=legend_elements, bbox_to_anchor=(1.05, 1), loc='upper left')

    fig.tight_layout()
    plt.savefig('./Plottings/heatmap_{}_W_{}_a_{}_b_{}_R_{}_P_{}_a_{}_b_{}_c_{}_f_{}_r_{}_p{}_ir{}.pdf'.format(player,W_bar,str(alpha)[-1],str(beta)[-1],str(R)[-1],str(P)[-1],a,b,c_b,f,str(r)[-1],str(p)[-1],str(ir)[-1]))
    return plt.show()
    

In [ ]:
#Plot the path to the steady state


def plot_path_to_steady_state(matrix,results,obs,paths):

    fig, ax = plt.subplots(2,2,figsize = (10,5))


    ax[0,0].grid(True)
    ax[0,0].set_ylabel('Wealth', color= 'black')  # we already handled the x-label with ax1
    ax[0,0].set_ylim(0, W_bar + 1 )
    ax[0,0].set_xlim(0, obs)

    ax[1,0].grid(True)
    ax[1,0].set_xlabel('Time')
    ax[1,0].set_ylabel('Corruption')
#    ax[1].tick_params(axis='y', labelcolor='tab:red')
    ax[1,0].set_ylim(0, W_bar*2)
    ax[1,0].set_xlim(0, obs)
                   
    ax[0,1].grid(True)
    ax[0,1].set_xlabel('Time')
    ax[0,1].set_ylabel('Liability')
    ax[0,1].set_ylim(0,L_bar +1)
    ax[0,1].set_xlim(0, obs)
    
    
    ax[1,1].grid(True)
    ax[1,1].set_xlabel('Time')
    ax[1,1].set_ylabel('Consumption')
    ax[1,1].set_ylim(0, W_bar +1)
    ax[1,1].set_xlim(0, obs)               
    
    for i in range(paths):
        path_X, path_D, acc = get_paths(matrix, results,obs)
        ax[0,0].plot(np.arange(obs),  path_X[:,0], label = 'Wealth', color='tab:blue', alpha = 0.2)
        ax[1,0].plot(np.arange(obs),  acc , label = 'Corruption', color='tab:red', alpha = 0.2)
        ax[0,1].plot(np.arange(obs),  path_X[:,1], label = 'Liability', color='tab:orange', alpha = 0.2)
        ax[1,1].plot(np.arange(obs), path_X[:,0] - path_D[:,1], label = 'Consumption', color='tab:green', alpha = 0.2)
        
    
    

    #ax[0].legend(loc = 'upper left')
    #ax[1].legend(loc = 'upper left')
    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    plt.savefig('./Plottings/path_{}_W_{}_a_{}_b_{}_R_{}_P_{}_a_{}_b_{}_c_{}_f_{}_r_{}_p{}_ir{}.pdf'.format(player,W_bar,str(alpha)[-1],str(beta)[-1],str(R)[-1],str(P)[-1],a,b,c_b,f,str(r)[-1],str(p)[-1],str(ir)[-1]))
    
    return plt.show()


In [ ]:
###### GO CAREFULL OVER TTHE STEPS AND PROPERLY COMMENT ###########
#Fix the TRY and EXCEPTS ####

def states_sampler(states, matrix, matrix_2, results, results_2):
    try:
        di = get_di(results.sigma, matrix.X_unique, [states[0],states[2],states[4],states[5]], matrix.D_unique)
        wi = get_wi(results.sigma, matrix.X_unique, [states[0],states[2],states[4],states[5]], matrix.D_unique)
        di_j = get_dj('payer',states[4],100, states[0],states[2],states[4])

        dj = get_di(results_2.sigma, matrix_2.X_unique, [states[1],states[3],states[5],states[4]], matrix_2.D_unique)
        wj = get_wi(results_2.sigma, matrix_2.X_unique, [states[1],states[3],states[5],states[4]], matrix_2.D_unique)
        dj_j = get_dj('receiver',states[5],100, states[1],states[3],states[5])
    except:
        di = get_di(results_2.sigma, matrix_2.X_unique, [states[0],states[2],states[4],states[5]], matrix_2.D_unique)
        wi = get_wi(results_2.sigma, matrix_2.X_unique, [states[0],states[2],states[4],states[5]], matrix_2.D_unique)
        di_j = get_dj('payer',states[4],100, states[0],states[2],states[4])

        dj = get_di(results.sigma, matrix.X_unique, [states[1],states[3],states[5],states[4]], matrix.D_unique)
        wj = get_wi(results.sigma, matrix.X_unique, [states[1],states[3],states[5],states[4]], matrix.D_unique)
        dj_j = get_dj('receiver',states[5],100, states[1],states[3],states[5])

    if len(get_Si_prime(states[4],di,dj)) == 1:
        Si_prime = get_Si_prime(states[4],di,dj)[0]
    else:
        prob = get_prob(get_Si_prime(states[4],di,dj)[0], (get_Sj_prime(states[4], di, dj,get_Si_prime(states[4],di,dj)[0]) ) )
        obs_prob = np.random.binomial(1, prob, 1)[0]
        if obs_prob == 1:
            Si_prime = get_Si_prime(states[4],di,dj)[0]
        else:
            Si_prime = get_Si_prime(states[4],di,dj)[1]
    Sj_prime = get_Sj_prime(states[4],di,dj,Si_prime)
    Li_prime = get_Li_prime(states[2],di,dj,Si_prime)
    Lj_prime = Li_prime
    Wi_prime = get_Wi_prime(player, wi,Li_prime , Si_prime, Sj_prime)
    Wj_prime = get_Wi_prime(get_otherplayer(player), wj, Lj_prime, Sj_prime, Si_prime)
    try:
        wi_prime = get_wi(results.sigma, matrix.X_unique, [Wi_prime,Li_prime,Si_prime,Sj_prime], matrix.D_unique)
        wj_prime = get_wi(results_2.sigma, matrix_2.X_unique, [Wj_prime,Lj_prime,Sj_prime,Si_prime], matrix_2.D_unique)
    except:
        wi_prime = get_wi(results_2.sigma, matrix_2.X_unique, [Wi_prime,Li_prime,Si_prime,Sj_prime], matrix_2.D_unique)
        wj_prime = get_wi(results.sigma, matrix.X_unique, [Wj_prime,Lj_prime,Sj_prime,Si_prime], matrix.D_unique)        

    return np.array([Wi_prime,Wj_prime,Li_prime,Lj_prime, Si_prime, Sj_prime,wi_prime,wj_prime])


In [4]:
import scipy.stats as stats

def vector_sampler(obs,periods,matrix,matrix_2,results,results_2,mean,std): 
    probs = stats.norm.pdf(np.linspace( - W_bar/2 ,  W_bar/2, W_bar +1 ), mean, std) # Get the vector with pdf ordered and centered at the median
    vec = np.zeros([obs*periods,8],dtype = np.int8)
    i = 0
    row = 0
    while i < obs:
        j = 0
        states = [0,0,0,0,0,0,0,0]
        states[0] = np.random.choice(W_space, 1, p = probs)[0]
        states[1] = states[0]
        while j < periods:
            states = states_sampler(states, matrix, matrix_2, results, results_2)
            vec[row] = states
            if not ( [states[0],states[2],states[4],states[5]] in matrix.X_unique[:,:4].tolist() and [states[1],states[3],states[5],states[4]] in matrix_2.X_unique[:,:4].tolist()):
                j = 1000
            else:
                j += 1
            row +=1
        i +=1
    return vec

In [2]:
def histogram_stationary(results,matrix):
    '''
    Build a the bin for the plot function for the instance from SimpleCorruption() and a title strig .
    
    '''
    stationary1 = results.mc.stationary_distributions
    
    #Stationary equilibrium

    raw1 = []

    for i in range(len(stationary1[0])):
        if stationary1[0,i] != 0:
            raw1.append([stationary1[0,i],matrix.X_unique[i]])
            
    


    #Create frequency of W

    W_dist1 = np.zeros(W_bar + 1)

    for i in range(len(raw1)):
        for j in range(W_bar + 1):
            if raw1[i][1][0] == j:
                W_dist1[j] = W_dist1[j] + raw1[i][0]
                

    #Create the state S frequency

    S_dist1 = np.zeros(len(S_space))

    for i in range(len(raw1)):
        for j in range(len(S_space)):
            if raw1[i][1][2] == S_space[j]:
                S_dist1[j] = S_dist1[j] + raw1[i][0]
                
    space_dict = ['No Corruption', 'Corruption','Detected', 'Desisted','Convicted','Acquitted','Self-Rported', 'Plead Guilty']

    fig, ax = plt.subplots(2, 1, figsize=(10,6))

    ax[0].bar( W_space, W_dist1, 1, align='center',color = 'steelblue', zorder=3)
    #ax.grid(True,alpha=0.5, linestyle='--',zorder=0)
    ax[0].set_xticks(W_space)
    ax[0].set_xticklabels(W_space)
    ax[0].set_ylim(bottom= 0 , top = 1)
    ax[0].set_xlabel('Wealth')   
    ax[0].set_ylabel('Frequency')
#    ax[0].set_title('Stationary Strategy')

    

    ax[1].bar( np.arange(len(S_space)),S_dist1, 1, align='center',color = 'steelblue', zorder=3)
    #ax.grid(True,alpha=0.5, linestyle='--',zorder=0)
    ax[1].set_xticks(S_space)
    ax[1].set_xticklabels(space_dict)
    ax[1].set_ylim(bottom= 0 , top = 1)
    ax[1].set_xlabel('States of the World')   
    ax[1].set_ylabel('Frequency')
#    ax[1].set_title(title)
    
    fig.tight_layout()
    plt.savefig('./Plottings/hist_stat_{}_W_{}_a_{}_b_{}_R_{}_P_{}_a_{}_b_{}_c_{}_f_{}_r_{}_p{}_ir{}.pdf'.format(player,W_bar,str(alpha)[-1],str(beta)[-1],str(R)[-1],str(P)[-1],a,b,c_b,f,str(r)[-1],str(p)[-1],str(ir)[-1]))
    plt.show()

In [5]:
def histogram_sampling(vec,title):
    '''
    Build a the bin for the plot function for the instance from SimpleCorruption() and a title strig .
    
    '''
    
    #refactor - Perhaps it is overcomplicated#
    
    hist, count = np.unique(vec[:,6],return_counts = True)
    
    j = 0
    for i in W_space:
        if hist[j] != W_space[i]:
            hist = np.append(hist,W_space[i]) # fill the histogram with the missing values in W_space
            count = np.append(count,0) # complete with a 0 count the missing state
        elif len(hist) -1 > j:
            j +=1
        elif len(hist) >= len(W_space):
            j += 1
            hist = np.append(hist,W_space[j]) # fill the histogram with the missing values in W_space
            count = np.append(count,0) # complete with a 0 count the missing state
        
        
    W_dist1 = count[np.argsort(hist)] #return order of the count in accordance to each state position in sequence  
    W_dist1 = (W_dist1)/len(vec)
    
    hist, count = np.unique(vec[:,7],return_counts = True)
    
    j = 0
    for i in W_space:
        if hist[j] != W_space[i]:
            hist = np.append(hist,W_space[i]) # fill the histogram with the missing values in W_space
            count = np.append(count,0) # complete with a 0 count the missing state
        elif len(hist) -1 > j:
            j +=1
        elif len(hist) >= len(W_space):
            j += 1
            hist = np.append(hist,W_space[j]) # fill the histogram with the missing values in W_space
            count = np.append(count,0) # complete with a 0 count the missing state
    
    W_dist2 = count[np.argsort(hist)] #return order of the count in accordance to each state position in sequence  
    W_dist2 = (W_dist2)/len(vec)
    
    
    
    hist, count = np.unique(vec[:,4],return_counts = True)
    
    
    j = 0
    for i in range(len(S_space)):
        if hist[j] != S_space[i]:
            hist = np.append(hist,S_space[i]) # fill the histogram with the missing values in W_space
            count = np.append(count,0) # complete with a 0 count the missing state
        elif j == len(hist) - 1:
            pass
        else:
            j +=1 

    S_dist1 = count[np.argsort(hist)] #return order of the count in accordance to each state position in sequence  
    S_dist1 = (S_dist1)/len(vec)
    
    hist, count = np.unique(vec[:,5],return_counts = True)    
    
    j = 0
    for i in range(len(S_space)):
        if hist[j] != S_space[i]:
            hist = np.append(hist,S_space[i]) # fill the histogram with the missing values in W_space
            count = np.append(count,0) # complete with a 0 count the missing state
        elif j == len(hist) - 1:
            pass
        else:
            j +=1 

    S_dist2 = count[np.argsort(hist)] #return order of the count in accordance to each state position in sequence  
    S_dist2 = (S_dist2)/len(vec)
    
    
    space_dict = ['No Corruption', 'Corruption','Detected', 'Desisted','Convicted','Acquitted','Self-Rported', 'Plead Guilty']

    fig, ax = plt.subplots(2, 1, figsize=(10,6))

    ax[0].bar(W_space + 0.25, W_dist1, 0.5, align='center',color = 'steelblue', zorder=3)
    ax[0].bar(W_space - 0.25, W_dist2, 0.5 , align='center',color = 'tab:orange', zorder=3)
    #ax.grid(True,alpha=0.5, linestyle='--',zorder=0)
#    ax[0].set_xticks(W_space)
#    ax[0].set_xticklabels(W_space)
    ax[0].set_ylim(bottom= 0 , top = 1)
    ax[0].set_xlabel('Wealth')   
    ax[0].set_ylabel('Frequency')
    ax[0].legend(['Payer', 'Receiver'])
#    ax[0].set_title(title)


    rects1 = ax[1].bar( np.arange(len(S_space)) + 0.25,S_dist1, 0.5, align='center',color = 'steelblue', zorder=3)
    rects2 = ax[1].bar( np.arange(len(S_space)) - 0.25,S_dist2, 0.5, align='center',color = 'tab:orange', zorder=3)
    #ax.grid(True,alpha=0.5, linestyle='--',zorder=0)
    ax[1].set_xticks(S_space)
    ax[1].set_xticklabels(space_dict)
    ax[1].set_ylim(bottom= 0 , top = 1.1)
    ax[1].set_xlabel('States of the World')   
    ax[1].set_ylabel('Frequency')
    ax[1].bar_label(rects1, padding=3)
    ax[1].bar_label(rects2, padding=3)
#    ax[1].set_title(title)
    
    fig.tight_layout()
    plt.savefig('./Plottings/hist_sample_{}_W_{}_a_{}_b_{}_R_{}_P_{}_a_{}_b_{}_c_{}_f_{}_r_{}_p{}_ir{}.pdf'.format(title,W_bar,str(alpha)[-1],str(beta)[-1],str(R)[-1],str(P)[-1],a,b,c_b,f,str(r)[-1],str(p)[-1],str(ir)[-1]))
    plt.show()
    
    

In [4]:
def histogram_sampling_savings(vec,title):
    '''
    Build a the bin for the plot function for the instance from SimpleCorruption() and a title strig .
    
    '''
    hist, count = np.unique(vec[:,6],return_counts = True)
    
    j = 0
    for i in W_space:
        if hist[j] != W_space[i]:
            hist = np.append(hist,W_space[i]) # fill the histogram with the missing values in W_space
            count = np.append(count,0) # complete with a 0 count the missing state
        elif len(hist) -1 > j:
            j +=1
        elif len(hist) >= len(W_space):
            j += 1
            hist = np.append(hist,W_space[j]) # fill the histogram with the missing values in W_space
            count = np.append(count,0) # complete with a 0 count the missing state
        
        
    W_dist1 = count[np.argsort(hist)] #return order of the count in accordance to each state position in sequence  
    W_dist1 = (W_dist1)/len(vec)
    
    hist, count = np.unique(vec[:,7],return_counts = True)
    
    j = 0
    for i in W_space:
        if hist[j] != W_space[i]:
            hist = np.append(hist,W_space[i]) # fill the histogram with the missing values in W_space
            count = np.append(count,0) # complete with a 0 count the missing state
        elif len(hist) -1 > j:
            j +=1
        elif len(hist) >= len(W_space):
            j += 1
            hist = np.append(hist,W_space[j]) # fill the histogram with the missing values in W_space
            count = np.append(count,0) # complete with a 0 count the missing state
    
    W_dist2 = count[np.argsort(hist)] #return order of the count in accordance to each state position in sequence  
    W_dist2 = (W_dist2)/len(vec)
    
    

    fig, ax = plt.subplots(1, 1, figsize=(10,3))

    ax.bar(W_space + 0.25, W_dist1, 0.5, align='center',color = 'steelblue', zorder=3)
    ax.bar(W_space - 0.25, W_dist2, 0.5 , align='center',color = 'tab:orange', zorder=3)
    #ax.grid(True,alpha=0.5, linestyle='--',zorder=0)
#    ax[0].set_xticks(W_space)
#    ax[0].set_xticklabels(W_space)
    ax.set_ylim(bottom= 0 , top = 1)
    ax.set_xlabel('Savings')   
    ax.set_ylabel('Frequency')
    ax.legend(['Payer', 'Receiver'])
#    ax.set_title(title)

    fig.tight_layout()
    plt.savefig('./Plottings/hist_sample_savings{}_W_{}_a_{}_b_{}_R_{}_P_{}_a_{}_b_{}_c_{}_f_{}_r_{}_p{}_ir{}.pdf'.format(title,W_bar,str(alpha)[-1],str(beta)[-1],str(R)[-1],str(P)[-1],a,b,c_b,f,str(r)[-1],str(p)[-1],str(ir)[-1]))
    plt.show()
    
    

In [3]:
def get_IC_star(R_star,P_star):
    n = 100
    x = np.arange(0.0000001, 1,1/n )
    N = 0.5

    alphaIE = []
    for i in x:
        alphaIE.append(((-b/(gamma**2)) + a)/(i*(a+f))) #

    alphaIE = np.asarray(alphaIE)      

    alphaIB = []

    for i in x:
        alphaIB.append(((-c_b/(gamma**2)) + b)/ (i*(b+f)))  #

    alphaIB = np.asarray(alphaIB)   

    #Ploting Indiference curves

    R = []
    for i in x:
        R.append((((gamma**2)*a) +( R_star*f))/(((gamma**2)*(a+f))*i)) #

    R = np.asarray(R)  


    P = [((gamma*a) +( P_star*f))/(gamma*(a+f))]*n

    lim = int(((gamma*a) +( P_star*f))/(gamma*(a+f))*100)

    P = np.asarray(P)  

    fig, ax = plt.subplots(figsize = (5,4))

    ax.plot(x, alphaIE, linewidth=2)
    ax.plot(x, alphaIB,'--', linewidth=2)
    ax.plot(x,R,'-.',linewidth=.5 )
    ax.plot(P, x,linewidth=.5 )

    fig.add_subplot(ax)

    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

    fig.text(0.92, 0.05, '$𝛽$')
    fig.text(0.1, 0.9, '$𝛼$')
    #fig.text(0.65, 0.08, '$𝛽~(P*) $')
    #fig.text(0.02, 0.485, '$𝛼~(R*)$')

    ax.fill_between(x[0:lim + 1], alphaIE[0:lim + 1], facecolor='salmon', alpha=0.1)
    ax.fill_between(x[lim:100], alphaIE[lim:100], facecolor='green', alpha=0.1)
    ax.fill_between( x, alphaIE , R ,facecolor='white', alpha=1,label='_nolegend_')
    ax.fill_between(x, alphaIE, R, facecolor='yellow', alpha=0.1)

    plt.ylim([0, 1])
    plt.xlim([0, 1])
    plt.grid(True)
    plt.xticks(np.arange(0,1.1,.1))
    plt.yticks(np.arange(0,1.1,.1))
    #plt.title('Indiference curves')
    plt.legend(('IC Payer', 'IC Receiver','Self-Reporting Bonus','Plea Agreement', 'Corruption Zone','Plea Bragaining','Self-Reporting'), bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.savefig('./Plottings/ICSTAR_{}_W_{}_a_{}_b_{}_R_{}_P_{}_a_{}_b_{}_c_{}_f_{}_r_{}_p{}_ir{}.pdf'
                .format(player,W_bar,str(alpha)[-1],str(beta)[-1],str(R)[-1],str(P)[-1],a,b,c_b,f,str(r)[-1],str(p)[-1],str(ir)[-1]))
    plt.show()

In [4]:
def get_ICs():
    n = 100
    x = np.arange(0.0000001, 1,1/n )
    N = 0.5

    alphaIE = []
    for i in x:
        alphaIE.append(((-b/(gamma**2)) + a)/ (i*(a+f))) #

    alphaIE = np.asarray(alphaIE)      

    alphaIB = []

    for i in x:
        alphaIB.append(((-c_b/(gamma**2)) + b)/ (i*(b+f)))  #

    alphaIB = np.asarray(alphaIB)   

    #Ploting Indiference curves

    fig, ax = plt.subplots(figsize = (5,4))

    ax.plot(x, alphaIE, linewidth=2)
    ax.plot(x, alphaIB,'--', linewidth=2)

    fig.add_subplot(ax)

    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

    fig.text(0.92, 0.05, '$𝛽$')
    fig.text(0.11, 0.9, '$𝛼$')

    ax.fill_between(x, alphaIE, facecolor='salmon', alpha=0.1)

    plt.ylim([0, 1])
    plt.xlim([0, 1])
    plt.grid(True)
    plt.xticks(np.arange(0,1.1,.1))
    plt.yticks(np.arange(0,1.1,.1))
    #plt.title('Indiference curves')
    plt.legend(('IC Payer', 'IC Receiver', 'Corruption Zone' ),
               loc='best', shadow=False)
    #plt.grid(True)
    # fig.text(0.18, 0.2, 'Corruption')
    # fig.text(0.5, 0.4, 'No Corruption')
    plt.savefig('./Plottings/ICS_{}_W_{}_a_{}_b_{}_R_{}_P_{}_a_{}_b_{}_c_{}_f_{}_r_{}_p{}.pdf'.format(player,W_bar,str(alpha)[-1],str(beta)[-1],str(R)[-1],str(P)[-1],a,b,c_b,f,str(r)[-1],str(p)[-1]))
    plt.show()


In [ ]:
def get_scenario_series(sample):
    ''' PLace the sample from the pre regime, and have the matrices and results from the second regime on the golbals'''
    Corruption = np.zeros(10)
    Detection = np.zeros(10)
    Corruption[0] = 1

    #Before Change in the Regime:
    for i in range(4):
        Corruption[i] = get_distributions(sample)[1][1]
        Detection[i] = get_distributions(sample)[1][2] + get_distributions(sample)[1][6] + get_distributions(sample)[1][7] 

    #After Change in the Regime
    for i in range(3,10):
        sample = get_state_distribution(400,5,matrix , matrix_2, results, results_2, 0 ,1,sample)
        Corruption[i] = get_distributions(sample)[1][1]
        Detection[i] = get_distributions(sample)[1][2] + get_distributions(sample)[1][6] + get_distributions(sample)[1][7] 
    return Corruption, Detection
    
    

In [1]:
### Plot the Path

def plot_scenario(title, Corruption, Detection):
    fig, ax = plt.subplots(figsize = (9,3))

    x = np.linspace(0, 10, 10)
    y = np.linspace(0, 1, 100)
    intervention = np.ones(10) * 2.2

    # Using set_dashes() to modify dashing of an existing line
    ax.plot(x, Corruption, label='Corruption')
    ax.plot(x, Detection, dashes=[6, 2], label='Detection')
    ax.plot(intervention,x , color= 'red', label = 'Change in Regime' )



    ax.set_ylim(0,1)
    ax.legend()
    ax.set_title(title)

    fig.savefig('./Plottings/scenario_{}.pdf'.format(title))
    
    plt.show()



In [ ]:
def get_distributions(sample):
    ''' Gets the distribution of states in a vector '''

    hist, count = np.unique(sample[:,2],return_counts = True)

    j = 0
    if len(hist) != len(L_space):
        for i in L_space:
            if hist[j] != L_space[i]:
                hist = np.append(hist,L_space[i]) # fill the histogram with the missing values in W_space
                count = np.append(count,0) # complete with a 0 count the missing state
            elif len(hist) -1 > j:
                j +=1
            elif len(hist) >= len(L_space):
                j += 1
                hist = np.append(hist,L_space[j]) # fill the histogram with the missing values in W_space
                count = np.append(count,0) # complete with a 0 count the missing state

    L_dist = count[np.argsort(hist)] #return order of the count in accordance to each state position in sequence  
    L_dist = (L_dist)/len(sample[:,2])
    
    
    hist, count = np.unique(sample[:,4],return_counts = True)
    
    j = 0
    for i in range(len(S_space)):
        if hist[j] != S_space[i]:
            hist = np.append(hist,S_space[i]) # fill the histogram with the missing values in W_space
            count = np.append(count,0) # complete with a 0 count the missing state
        elif j == len(hist) - 1:
            pass
        else:
            j +=1 

    S_dist = count[np.argsort(hist)] #return order of the count in accordance to each state position in sequence  
    S_dist = (S_dist)/len(sample)
    return L_dist, S_dist


In [ ]:


def get_state_distribution(obs,periods,matrix,matrix_2,results,results_2,mean,std,sample): 
    '''From the pre intervention sample, derives the path of the pre and post intervention'''
    probs = stats.norm.pdf(np.linspace( - W_bar/2 ,  W_bar/2, W_bar +1 ), mean, std) # Get the vector with pdf ordered and centered at the median
    L_dist , S_dist = get_distributions(sample) 
    vec = np.zeros([obs*periods,8],dtype = np.int8)
    i = 0
    row = 0
    while i < obs:
        j = 0
        state_condition = True
        while state_condition: #it runs until finding a scenario that is feasible
            states = [0,0,0,0,0,0,0,0]
            states[0] = np.random.choice(W_space, 1, p = probs)[0]
            states[1] = states[0]
            states[2] = np.random.choice(L_space, 1, p = L_dist)[0]
            states[3] = states[2]
            states[4] = np.random.choice(S_space, 1, p = S_dist)[0]
            states[5] = states[4]
            state_condition = not([states[0],states[2],states[4],states[5]] in matrix.X_unique[:,:4].tolist() and [states[1],states[3],states[5],states[4]] in matrix_2.X_unique[:,:4].tolist())
        while j < periods:
            states = states_sampler(states, matrix, matrix_2, results, results_2)
            vec[row] = states
            if not ( [states[0],states[2],states[4],states[5]] in matrix.X_unique[:,:4].tolist() and [states[1],states[3],states[5],states[4]] in matrix_2.X_unique[:,:4].tolist()):
                j = 1000
            else:
                j += 1
            row +=1
        i +=1
    return vec